In [16]:
import secret as k
import requests
import json
import csv

In [17]:
request_URL = 'http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/3772' # the specific location for HEATHROW
#request_URL = 'http://datapoint.metoffice.gov.uk/public/data/val/wxobs/all/json/all' # get everything. Note the JSON schema changes....
request_params = {
  'res' : 'hourly',
  'key' : k.api_key
}
response = requests.get(request_URL, request_params)
print(response.status_code)

200


In [49]:
def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=2)
    print(text)

#jprint(response.json())

In [47]:
incoming_locations = []
incoming_observations = []
incoming_list_to_parse = []

type(response.json()['SiteRep']['DV'])

if (isinstance(response.json()['SiteRep']['DV']['Location'], dict)) :
  incoming_list_to_parse.append(response.json()['SiteRep']['DV']['Location'])
else :
  incoming_list_to_parse = response.json()['SiteRep']['DV']['Location']

for location in incoming_list_to_parse:
  raw_observations = location.pop('Period', None)
  incoming_locations.append(location)
  # print(location['name'])

  current_location_id = location['i']
  observations_to_parse = []

  if (isinstance(raw_observations, dict)) :
    observations_to_parse.append(raw_observations)
  else :
    observations_to_parse = raw_observations

  for observation_period in observations_to_parse :
    observation_date = observation_period['value']

    reps_to_parse = []
    if (isinstance(observation_period['Rep'], dict)) :
      reps_to_parse.append(raw_observations)
    else :
      reps_to_parse = observation_period['Rep']

    for rep in reps_to_parse:
      rep['LocationID'] = current_location_id
      rep['ObservationDate'] = observation_date
      incoming_observations.append(rep)

#incoming_locations
#incoming_observations

In [53]:
# outputs a list of dicts to a file,
# using the keys of the first dict as the header row
# this errors if the others have additional fields
def list_to_csv(list_of_dicts, fieldnames, fname):
  with open(fname, 'w', newline='') as out_file:
    output = csv.DictWriter(out_file, fieldnames=fieldnames)

    output.writeheader()
    for row in list_of_dicts:
      output.writerow(row)


list_to_csv(incoming_locations, list(incoming_locations[0].keys()), 'incoming_locations.csv')
observation_fieldnames = ["G","T","V","D","S","W","P","Pt","Dp","H",'LocationID','ObservationDate','$']
list_to_csv(incoming_observations, observation_fieldnames, 'incoming_observations.csv')